In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Osteoarthritis/GSE142049'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional data of inflamatory arthritis B cells"
!Series_summary	"With a focus on rheumatoid arthritis (RA), we sought new insight into genetic mechanisms of adaptive immune dysregulation to help prioritise molecular pathways for targeting in this and related immune pathologies. Whole genome methylation and transcriptional data from isolated CD4+ T cells and B cells of >100 genotyped and phenotyped inflammatory arthritis patients, all of whom were naïve to immunomodulatory treatments, were obtained. Analysis integrated these comprehensive data with GWAS findings across IMDs and other publically available resources."
!Series_overall_design	"Suspected inflammatory arthritis patients of Northern European ancestry were recruited prior to treatment with immunomodulatory drugs. RA patients were classified using current, internationally accepted criteria, and matched with disease controls in respect of demographic and clinical characteristics. CD19

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check gene expression data availability
is_gene_available = True  # Based on information, it mentions transcriptional data

# Identify the rows for the variables
trait_row = 6  # Based on the dictionary, 'working_diagnosis' contains 'Osteoarthritis'
age_row = 2  # The dictionary contains an 'age' field
gender_row = 1  # The dictionary contains a 'gender' field

# Define the conversion functions
def convert_trait(value):
    diagnosis = value.split(":")[1].strip()
    return 1 if diagnosis == 'Osteoarthritis' else 0

def convert_age(value):
    try:
        return float(value.split(":")[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(":")[1].strip()
    return 1 if gender == 'M' else 0

# Save cohort info
save_cohort_info('GSE142049', './preprocessed/Osteoarthritis/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Osteoarthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Osteoarthritis/trait_data/GSE142049.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4218229': [0.0, 82.0, 1.0], 'GSM4218230': [0.0, 29.0, 1.0], 'GSM4218231': [0.0, 61.0, 0.0], 'GSM4218232': [0.0, 56.0, 1.0], 'GSM4218233': [0.0, 55.0, 0.0], 'GSM4218234': [0.0, 72.0, 1.0], 'GSM4218235': [0.0, 50.0, 0.0], 'GSM4218236': [0.0, 22.0, 1.0], 'GSM4218237': [0.0, 61.0, 1.0], 'GSM4218238': [0.0, 53.0, 0.0], 'GSM4218239': [0.0, 54.0, 0.0], 'GSM4218240': [0.0, 20.0, 0.0], 'GSM4218241': [0.0, 49.0, 0.0], 'GSM4218242': [0.0, 61.0, 1.0], 'GSM4218243': [1.0, 53.0, 0.0], 'GSM4218244': [0.0, 59.0, 1.0], 'GSM4218245': [0.0, 56.0, 0.0], 'GSM4218246': [0.0, 57.0, 1.0], 'GSM4218247': [0.0, 55.0, 1.0], 'GSM4218248': [0.0, 56.0, 0.0], 'GSM4218249': [0.0, 35.0, 0.0], 'GSM4218250': [0.0, 58.0, 0.0], 'GSM4218251': [0.0, 69.0, 1.0], 'GSM4218252': [0.0, 46.0, 0.0], 'GSM4218253': [1.0, 57.0, 0.0], 'GSM4218254': [0.0, 50.0, 0.0], 'GSM4218255': [0.0, 66.0, 0.0], 'GSM4218256': [0.0, 52.0, 0.0], 'GSM4218257': [0.0, 63.0, 0.0], 'GSM4218258': [0.0, 51.0, 1.0], 'GSM4218259': [0.0, 50.0, 0.0], 'GSM421

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index([], dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = (True)


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Define the columns for identifiers and gene symbols based on the preview
    identifier_key = 'ID'
    gene_symbol_key = 'Symbol'

    # 2. Get the dataframe storing the mapping between probe IDs and genes
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
    gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Osteoarthritis/gene_data/GSE142049.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Osteoarthritis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE142049', './preprocessed/Osteoarthritis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Osteoarthritis/GSE142049.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
A new JSON file was created at: ./preprocessed/Osteoarthritis/cohort_info.json
